# Demo: Defining Control_M Workflows using Python

# Step 1 - Setup

## Step 1A - Install the library

In [1]:
!pip --version

pip 21.2.4 from /opt/conda/lib/python3.8/site-packages/pip (python 3.8)


In [2]:
!pip install git+https://github.com/tadinve/ctm_python_client.git

  Cloning https://github.com/tadinve/ctm_python_client.git to /tmp/pip-req-build-eo5m_fna
  Running command git clone -q https://github.com/tadinve/ctm_python_client.git /tmp/pip-req-build-eo5m_fna
  Resolved https://github.com/tadinve/ctm_python_client.git to commit bd05e8e7c01ab68360caedb28494927ee8ee07c6


In [3]:
import os
from ctm_python_client.core.bmc_control_m import CmJobFlow
from ctm_python_client.jobs.dummy import DummyJob

# Step 2 - Instantiate, Authenticate and Schedule


## Step 2A - Define the Connection Variables

In [4]:
# Create a file .secrets with the following three lines. One for uri, one for user and one for password.
"""
https://acb-rhctmv20.centralus.cloudapp.azure.com:8443/automation-api
username
password
"""
BASE_PATH = os.path.abspath("")
with open(BASE_PATH + "/.secrets", "r") as fp:
    ctm_uri = fp.readline().strip()
    ctm_user = fp.readline().strip()
    ctm_pwd = fp.readline().strip()

## Step 2B - Create the object

In [5]:
from ctm_python_client.session.session import Session

session = Session(endpoint=ctm_uri, username=ctm_user, password=ctm_pwd)
session.get_token()

'17DCA30C597E54E07C3EC09D3FE4023C1F06B356A2E7FB5B89A89A2D924DC8849796A93D641484BBB323A1035A28A8988E12CD74E58E267F3FA5B5597DFBCACE'

In [6]:
t1_flow = CmJobFlow(
    application="Naga0.2_Demo", sub_application="Demo-02", session=session, ctm_uri=ctm_uri
)

TypeError: __init__() got an unexpected keyword argument 'ctm_uri'

## Step 2C - Define the Schedule

In [ ]:
t1_flow.set_run_as(username="ctmuser", host="acb-rhctmv20")

In [ ]:
# Define the schedule
months = ["JAN", "OCT", "DEC"]
monthDays = ["ALL"]
weekDays = ["MON", "TUE", "WED", "THU", "FRI"]
fromTime = "0300"
toTime = "2100"
t1_flow.set_schedule(months, monthDays, weekDays, fromTime, toTime)

# Step 3  - Create Folder

In [ ]:
# Create Fodler
f1 = t1_flow.create_folder(name="HelloWorld")

# Step 4 - Create Tasks

In [ ]:
start = t1_flow.add_job(f1, DummyJob(f1, "Start-Flow"))
end = t1_flow.add_job(f1, DummyJob(f1, "End-Flow"))

hello_world_id = t1_flow.add_job(f1, DummyJob(f1, "Hello-World"))

# Step 5 - Chain Tasks

In [ ]:
# start >>  hello_world_id >> end
t1_flow.chain_jobs(f1, [start, hello_world_id, end])

# Step 6 - Display Workflow

## Step 6A - Display DAG

In [ ]:
# View the t1_flow Details
nodes, edges = t1_flow.get_nodes_and_edges()
nodes, edges

In [ ]:
# display using graphviz
from ctm_python_client.utils.displayDAG import DisplayDAG

# sudo apt-get install graphviz (on unix)
#  or
# brew install graphviz (for mac)
DisplayDAG(t1_flow).display_graphviz()

## Step 6B - Display JSON

In [ ]:
t1_flow.display_json()

# Step 7 - Submit Workflow to Control-M

In [ ]:
t1_flow.deploy()

In [ ]:
t1_flow.run()